
# Download the Chrome Driver to Match your Brave Version:
https://chromedriver.chromium.org/downloads  
Check your chrome version in Brave (Options => 'About Brave')  
From the Chrome Driver URL above, download the version that matches.  (has the same first three digits, 'major')  
Extract the EXE to the root of this directory so the EXE is in the root.
  
Example, If you have version 108.0.5359.128, then download the 108.x.x.x driver.


# Objective:   
## This Notebook:
- Emulate opening/closing tabs   
- Emulate clicking UI controls   
- Emulate navigation (w/ randomness)   

Brave seems to have one final control outside these three organic HID (Human Interface Devices) movements.   
  

 ## Raspberry PI Pico/Circuit Python
Circuit python on a $4 raspberry pi pico can emulate HID behavior.  See the attached code.py script to install on your circuit python device for HID emulation.  There seems to be a trick w/ speed and direction. 
Demo for Circuit Python here: https://youtu.be/07vG-_CcDG0   


In [1]:
!pip install selenium
!pip install requests

# Imports

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException
from datetime import datetime

import requests
import os, subprocess, time, json, random

# Parameters

In [3]:
brave_install = r'C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe'
brave_user_data_dir= r'%appdata%\Local\BraveSoftware\Brave-Browser\User Data\Default'
brave_debug_port = '9222'

bot_parms = requests.get('https://hobbytimewith.me/targets.json').json()

verbose_output = False

# Constrect Launcher and Initalize

In [4]:
cmd = r'"{0}" --remote-debugging-port={1} --user-data-dir="{2}"'.format(
    brave_install, brave_debug_port, brave_user_data_dir)

f = open("launcher.bat", "w")
f.write(cmd)
f.close()

p = subprocess.Popen('launcher.bat', start_new_session=True)

# Connect w/ Driver

In [5]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")
driver = webdriver.Chrome(options=chrome_options)

## Test Nav

In [6]:
def nav(url):
    driver.get(url)
    time.sleep(5) # allow for pageload, do whatever you want
    return driver.title

In [7]:
title = nav("http://www.google.com")
print(f"Current Tab Open '{title}'")

Current Tab Open 'Google'


# Test Tab Control
## Approve popups int he browser from google.com!!!
## Approve popups int he browser from twitch.com!!!
Poping new tabs is a form of activity we are trying to emulate, the only way to do this via chromdriver is w/ a window.open command in JS. As a result presents itself as a popup. 

In [8]:
def popATab():
    driver.execute_script('''window.open("about:blank");''')  
    time.sleep(0.5)
    if len(driver.window_handles) > 1:
        for handle in driver.window_handles[1:]:
            driver.switch_to.window(handle) 
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

In [9]:
popATab()

# Click th Channel Points Button (for activity)

In [10]:
def getClickChannelPointsButton():
    while True:
        try:
            elements = driver.find_elements(By.XPATH, '//button')
            GetBitsElements = []
            for element in elements:
                cl1 = element.get_attribute("aria-label")
                if cl1 == "Claim Bonus":
                    GetBitsElements.append(element)
            if len(GetBitsElements) > 0:
                # Colect them points
                return GetBitsElements[0]
            else:
                return None
        except StaleElementReferenceException as e:
            if verbose_output:
                print(f'StaleElementReferenceException, Attempting Again...')

# Begin Cycling the URLs
After this is running, plug in your raspberry pi to emulate HID mouse movement.

In [11]:
while True:
    try:
        popATab()
        nav(random.choice(bot_parms['bot_urls']))
        seconds = random.randrange(
            bot_parms['bot_url_delay_range_min'], 
            bot_parms['bot_url_delay_range_max'])

        if verbose_output:
            print(f"sleeping {seconds} seconds")
        time.sleep(seconds)
        element = getClickChannelPointsButton()

        if element != None:
            if verbose_output:
                print("Clicking Channel Points Button")
            element.click()

        time.sleep(5)
    except Exception as e:
        print(f'{datetime.now().strftime("%m/%d/%Y %H:%M:%S")}: Exception: {e}')
        time.sleep(1)

KeyboardInterrupt: 